In [ ]:
pip install newsapi-python

In [22]:
import numpy as np
import pandas as pd
import datetime
import re
import json
import requests
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pickle

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# Don't run this cell, it only consists of 31 days data, fetch news data with news api
## set api_key for news api
newsapi = NewsApiClient(api_key='3486e577627249b2b78a602bf7ab4ebc')

## create the list of dates
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, 31)]
date_strings = [dt.strftime('%Y-%m-%d') for dt in date_list]

## import nltk pre-trained sentiment analyzer
sid = SentimentIntensityAnalyzer()

## fetch news and perform sentiment analysis on their descriptions
descriptions_date = {}
descriptions_polaritys = {}
descriptions_context = []

for date_str in date_strings:
    descriptions = []
    polaritys = []
    general_polarity = {'neg': 0.0, 'pos': 0.0}
    all_articles = newsapi.get_everything(q='bitcoin', from_param=date_str, to=date_str, language='en')
    for i in range(len(all_articles['articles'])):
        description = all_articles['articles'][i]['description']
        if description:
            non_symbols = re.sub(r'[^\w]', ' ', description) # remove symbols
            non_digits = re.sub(r'\d+', ' ', non_symbols) # remove digits
            one_space = re.sub(' +', ' ', non_digits) # remove extra whitespaces
            description_lower = one_space.lower() # transform characters into lower-case
            description_words = ' '.join(w for w in description_lower.split() if len(w)>1) # remove single characters
            descriptions.append(description_words) # save descriptions context
            polarity = sid.polarity_scores(description_words)
            #save descriptions polarity
            for polar in ['neg', 'pos']:
              general_polarity[polar] += polarity[polar]
            descriptions_context.append(description_words)
    descriptions_date[date_str] = descriptions
    descriptions_polaritys[date_str] = max(general_polarity, key=general_polarity.get)
descriptions_polaritys

In [234]:
# fetch data with ContextualWeb api
## create the list of dates
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, 220)]
date_strings = [dt.strftime('%Y-%m-%d') for dt in date_list]

## import nltk pre-trained sentiment analyzer
sid = SentimentIntensityAnalyzer()

## fetch news and perform sentiment analysis on their descriptions
descriptions_date = {}
descriptions_polaritys = {}
descriptions_context = []

url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"
querystring = {"autoCorrect":"false","pageNumber":"1","pageSize":"10","q":"Bitcoin","safeSearch":"false", "fromPublishedDate":"2020-1-1", "toPublishedDate":"2020-1-2"}
headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "6e64d32139msh66d110fec2ee5d6p1b2f8ajsn506563d39c4e"
    }

for idx in range(len(date_strings)-1):
    descriptions = []
    polaritys = []
    general_polarity = {'neg': 0.0, 'pos': 0.0}
    querystring['fromPublishedDate'] = date_strings[idx+1]
    querystring['toPublishedDate'] = date_strings[idx]
    response = requests.request("GET", url, headers=headers, params=querystring)
    response_json = response.json()['value']
    for i in range(len(response_json)):
        description = response_json[i]['description']
        if description:
            non_symbols = re.sub(r'[^\w]', ' ', description) # remove symbols
            non_digits = re.sub(r'\d+', ' ', non_symbols) # remove digits
            one_space = re.sub(' +', ' ', non_digits) # remove extra whitespaces
            description_lower = one_space.lower() # transform characters into lower-case
            description_words = ' '.join(w for w in description_lower.split() if len(w)>1) # remove single characters
            descriptions.append(description_words) # save descriptions context
            polarity = sid.polarity_scores(description_words)
            #save descriptions polarity
            for polar in ['neg', 'pos']:
              general_polarity[polar] += polarity[polar]
            descriptions_context.append(description_words)
    descriptions_date[date_strings[idx+1]] = descriptions
    descriptions_polaritys[date_strings[idx+1]] = max(general_polarity, key=general_polarity.get)

## save news data and their polarities
f = open("./drive/My Drive/final_project/file.pkl", "wb")
pickle.dump(descriptions_date, f)
f.close()

f = open("./drive/My Drive/final_project/polarities.pkl", "wb")
pickle.dump(descriptions_polaritys, f)
f.close()

['2020-01-31', '2020-01-30', '2020-01-29', '2020-01-28', '2020-01-27']